# Projeto 1 - Ciência dos Dados

Nome: Gabriela Kimi Katsurayama

Nome: João Pedro Rodrigues dos Santos

Nome: Luiza Ferreira Ehrenberger

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import nltk
import string
from nltk.corpus import stopwords
pd.set_option("display.max_colwidth", None)

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\jpthe\Ciencia de dados\CD21-2\P1 git\Cdados1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
filename = 'Iphone 12 novo.xlsx'

In [4]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,eu do nada me pego rindo do professor falando de burguesia e tirando um iphone 12 pro max do bolso kkkkk,0
1,"trocarias alguém da escola,por um iphone 12?\n-waweeee eliane.😭",0
2,"sobre o iphone 6s “não ter mais whatsapp”. é td mentira, a maioria ta no ios 12 pra cima, então deixem de alarme ate pq so vai sair se vc tiver no ios 9",3
3,ios 15 ja meu iphone n guenta nem o 12 direito https://t.co/mcshdy359r,4
4,nosso que pechincha 👀🙄uau 🙃\nhttps://t.co/jyhyn71jch,0


In [5]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)
# with open("Iphone 12 novo.xlsx", "r", encoding="utf-8-sig") as arquivo_texto:
#     test = arquivo_texto.read()

,Teste,Classificação
0,"queria lançar o iphone 12 mas é mto dinheiro numa coisa só, o certo é pegar o 11 que ai já da pra pegar agr",4.0
1,ganhei um iphone 12 pro max e o apple watch de presente 😍😍😍😍😍😍😍😍😍😍😍😍,0.0
2,@forsjin iphone? meu namorado pagou 1k no 12 fiquei em choque vius,1.0
3,tem uma criança que tá do meu lado que tem um iphone x e a mãe dele tem um 12 pro max. tô indignado com meu j7 todo quebrado,1.0
4,pensando que é tão besteira pra mim trocar meu iphone 11 pro 12 no natal,1.0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Para realizar o projeto, nós escolhemos um produto muito comentado e famoso, o Iphone. Pensando no enunciado da rúbrica, acabamos preferindo criar uma planilha com 1.252 tweets (treinamento + teste) e 5 classificações, com 0 sendo muito irrelevante e 5 muito relevante. Para filtrá-los optamos por colocar comentários negativos e pertinentes como 5 (Muito Relevantes) e comentários mais positivos como neutro ou irrelevante.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
from typing import Awaitable


def limpa_txt(txt):
  carac = ['\n', ':', '"', '`', '!', '?', ';','(',')', '[', ']', '%', '&', ',', '.', '@', '/']
  for i in txt:
    if i in carac:
        testador = txt.find(i)
        txt = txt[:testador] + txt[testador+1:]
  vn = arruma_esp(txt)
  return vn

def arruma_esp(txt):
  i = 0
  frase = ''
  aa = 0
  while i < len(txt)+1:
    if i >= 1:
        if i >= len(txt):
            frase += txt[i-1]
            return frase
        else:     
            if txt[i] == ' ' and txt[i-1] == ' ':
                aa += 1
            else:     
                frase += txt[i-1]
    i += 1


In [7]:
stop_words = stopwords.words("portuguese")
#wordnet = WordNetLemmatizer()
def text_preproc(x):
  #x = x.encode('ascii', 'ignore').decode()
  #a = re.sub(r'https*\S+', ' ', x)
  #b = re.sub(r'\s{2,}', ' ', a)
  c = re.sub(r'@\S+', ' ', x)
  d = re.sub(r'#\S+', ' ', c)
  e = re.sub(r'\'\w+', '', d)
  f = re.sub('[%s]' % re.escape(string.punctuation), ' ', e)
  #g = re.sub(r'\w*\d+\w*', '', f)
  h = f.strip('\n')
  return h


In [8]:
# qtd = train.groupby('Classificação').size().div(len(train))
# qtd

filtra_linhas_0 = train['Classificação'] == 0
mir = train.loc[filtra_linhas_0, :]

filtra_linhas_1 = train['Classificação'] == 1
ir = train.loc[filtra_linhas_1, :]

filtra_linhas_2 = train['Classificação'] == 2
n = train.loc[filtra_linhas_2, :]

filtra_linhas_3 = train['Classificação'] == 3
r = train.loc[filtra_linhas_3, :]

filtra_linhas_4 = train['Classificação'] == 4
mr = train.loc[filtra_linhas_4, :]


#/----------------------------------------------------/

m_i_r = mir['Treinamento'].to_string().lower()
m_i_r2 = limpa_txt(m_i_r)
m_i_r3 = m_i_r2.split()
serie_tweets = pd.Series(m_i_r3)
tabela_tts_mir_relativa = serie_tweets.value_counts(True)
tabela_tts_mir = serie_tweets.value_counts()

ir_1 = ir['Treinamento'].to_string().lower()
ir2 = limpa_txt(ir_1)
ir3 = ir2.split()
serie_tweets = pd.Series(ir3)
tabela_tts_ir_relativa = serie_tweets.value_counts(True)
tabela_tts_ir = serie_tweets.value_counts()

n_1 = n['Treinamento'].to_string().lower()
n2 = limpa_txt(n_1)
n3 = n2.split()
serie_tweets = pd.Series(n2)
tabela_tts_n_relativa = serie_tweets.value_counts(True)
tabela_tts_n = serie_tweets.value_counts()

# aaa = ''
# aaaa = ''
# for x in m_i:
#     aaa += text_preproc(x)
# for x in aaa:
#     x = re.sub(r'https*\S+', ' ', x)
#     x = re.sub(r'\s{2,}', ' ', x)
#     aaaa += x

r_1 = r['Treinamento'].to_string().lower()
r_2 = limpa_txt(r_1)
r_3 = r_2.split()
serie_tweets = pd.Series(r_3)
tabela_tts_r_relativa = serie_tweets.value_counts(True)
tabela_tts_r = serie_tweets.value_counts()
#m_i2_5 = ''
# for x in m_i:
#     x = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', x, flags=re.MULTILINE)
#     #x = re.sub("https*\S+", " ", x)
#     x = re.sub(r'\w*\d+\w*', '', x)
#     teste_12 = x.strip('\n')
#     m_i2_5 += teste_12

mr_1 = mr['Treinamento'].to_string().lower()
mr2 = limpa_txt(mr_1)
mr3 = mr2.split()
serie_tweets = pd.Series(mr3)
tabela_tts_mr_relativa = serie_tweets.value_counts(True)
tabela_tts_mr = serie_tweets.value_counts()



#print(len(m_i3))
# aaaa
# #m_i2_5




In [9]:
x = 'ios  ja meu iphone n guenta nem o  direito https://t.co/mcshdyr'

x = re.sub(r'https*\S+', ' ', x)
x = re.sub(r'\s{2,}', ' ', x)
x

'ios ja meu iphone n guenta nem o direito '

In [10]:
# for i in m_i:
#     i = re.sub("https*\S+", " ", i)

In [11]:
todos = [m_i_r2, ir2, n2, r_2, mr2]
i = 0
tabela_geral = ''
while i < len(todos):
    tabela_geral += todos[i]
    i += 1
i = 0
teste3 = 0
while i < len(todos):
    teste3 += len(todos[i])
    i += 1
print(teste3)

todas_palavras = tabela_geral.split()
serie_geral = pd.Series(todas_palavras)

tabela_geral_relativa = serie_geral.value_counts(True)
#tabela_geral_relativa
#print(len(todas_palavras))

93310


In [12]:
tt = test['Teste'].sample().to_string().lower()
print(tt)

tt10 = cleanup(tt)
xa = re.sub(r'\w*\d+\w*', '', tt10)
xb = re.sub("@\S+", " ", xa)
xc = re.sub("https*\S+", " ", xb)
xd = limpa_txt(xc)
print(xd)

375    eu amo esses vídeos sobre iphone 13 do pessoal jurando q a apple finalmente vai diminuir aquele entalhe na tela aí qnd chega no lançamento n diminuem porra nenhuma, foi a msm coisa no 12 kk
 eu amo esses vídeos sobre iphone do pessoal jurando q a apple finalmente vai diminuir aquele entalhe na tela aí qnd chega no lançamento n diminuem porra nenhuma foi a msm coisa no kk


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [13]:
frase_cortada = xd.split()

#print(tabela_tts_mir_relativa['exibida'])
print(frase_cortada)

['eu', 'amo', 'esses', 'vídeos', 'sobre', 'iphone', 'do', 'pessoal', 'jurando', 'q', 'a', 'apple', 'finalmente', 'vai', 'diminuir', 'aquele', 'entalhe', 'na', 'tela', 'aí', 'qnd', 'chega', 'no', 'lançamento', 'n', 'diminuem', 'porra', 'nenhuma', 'foi', 'a', 'msm', 'coisa', 'no', 'kk']


In [14]:
prob0 = len(m_i_r2)/teste3
prob1 = len(ir2)/teste3
prob2 = len(n2)/teste3
prob3 = len(r_2)/teste3
prob4 = len(mr2)/teste3
print(prob0,prob1,prob2,prob3,prob4)

0.6090236844925517 0.15689636694888007 0.09709570249705284 0.08648590719108348 0.0504983388704319


In [15]:
#calcula a prob de estar la
i = 0
probFraseDado0 = 1
probFraseDado1 = 1
probFraseDado2 = 1
probFraseDado3 = 1
probFraseDado4 = 1


while i < len(frase_cortada):
    if frase_cortada[i] not in tabela_tts_mir:
        P_palavra0 = 1
    else:
        P_palavra0 = tabela_tts_mir_relativa[frase_cortada[i]]
    
    if frase_cortada[i] not in tabela_tts_ir:
        P_palavra1 = 1
    else:
        P_palavra1 = tabela_tts_ir_relativa[frase_cortada[i]]
    
    if frase_cortada[i] not in tabela_tts_n:
        P_palavra2 = 1
    else:
        P_palavra2 = tabela_tts_n_relativa[frase_cortada[i]]
    
    if frase_cortada[i] not in tabela_tts_r:
        P_palavra3 = 1
    else:
        P_palavra3 = tabela_tts_r_relativa[frase_cortada[i]]
    
    if frase_cortada[i] not in tabela_tts_mr:
        P_palavra4 = 1
    else:
        P_palavra4 = tabela_tts_mr_relativa[frase_cortada[i]]
        
    probFraseDado0 *= P_palavra0
    probFraseDado1 *= P_palavra1
    probFraseDado2 *= P_palavra2
    probFraseDado3 *= P_palavra3
    probFraseDado4 *= P_palavra4
    i += 1
        

In [16]:
resp0 = probFraseDado0*prob0
resp1 = probFraseDado1*prob1
resp2 = probFraseDado2*prob2
resp3 = probFraseDado3*prob3
resp4 = probFraseDado4*prob4
resps = []
print(resp0,resp1,resp2,resp3,resp4)
resps.append(resp0)
resps.append(resp1)
resps.append(resp2)
resps.append(resp3)
resps.append(resp4)


if max(resps) == resp2:
    print('Neutro')
    resps.remove(resp2)
    if max(resps) == resp0:
        print('Muito irrelevante')
    if max(resps) == resp1:
        print('Irrelevante')
    if max(resps) == resp3:
        print('Relevante')
    if max(resps) == resp4:
        print('Muito Relevante')







1.4683581448930667e-79 1.2526772042331323e-61 0.09709570249705284 2.65086515247137e-50 3.637296808550911e-42
Neutro
Muito Relevante


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

[Cleaning Text Data with Python](https://towardsdatascience.com/cleaning-text-data-with-python-b69b47b97b76) **Para formatação do texto**